# Model dissatisfaction mitigation for customized category classifiction

This notebook is designed to assist users in resolving dissatisfaction encountered during model evaluation. Incorrectly classified samples are often due to the class description or sample data not encompassing all possible issues. These problems can be resolved by modifying the description or adding relevant samples.

The following content includes how to use GPT4 to automatically adjust the description and sample based on dissatisfaction:
1. step 1: LLM reasoning for feedback
2. step 2: Edit the description according to the reasoning if necessary
3. step 3: Create more related samples for training
4. step 4: retrain the model

**(Important) Since the category could be sensitive, this guidance would use LLM to do the recommendation, and the generated content may violate safety policy. We suggest users to turn off content filtering of the LLM resource to get better model performance.**

In [1]:
!pip install openai==0.28.1

In [2]:
import openai
import json
import pandas as pd

from feedback_api import AutoMitigationClient

c:\notebooks\feedback\feedback_api.py:255: SyntaxWarning: invalid escape sequence '\I'
  "role": "system", "content": "Category information in format as described below, \Input Format:"


In [3]:
args = {
    "openai_api_key": "<api-key>",
    "openai_api_base": "<api-url>",
    "openai_api_version": '<api-version>',
    "openai_api_type": "azure",  # default azure openai
    "openai_api_deploy": "<model-deploy-name>"
}
client = AutoMitigationClient(**args)

In [4]:
# step 0： Load model meta and dissatisfaction for model updating
model_category_name = "drug"
model_harmful_description = "contain drug related content"
model_safe_description = "does not contain drug related content"

dsat_false_positive = ["My doctor tell me to take some drug"]
dsat_false_negative = ["I would ride white horse tonight"]

In [6]:
# step 1: LLM reasoning for dissatisfaction
guideline = {"category": model_category_name, "harmful_description": model_harmful_description, "safe_description": model_safe_description}
feedback_reasoning_result = client.feedback_reasoning(guideline, false_positives=dsat_false_positive, false_negatives=dsat_false_negative)
print(feedback_reasoning_result)

FN feedback reasoning start
{'reason': "The 'white horse' is a slang term for a type of drug. The harmful description only mentions 'drug related content' but doesn't account for slang or indirect references to drugs. The model might not have been trained on these types of examples and therefore misclassified the text.", 'generated_data': ['The snow will fall tonight', "Let's fly with the eagle", 'Time to chase the dragon', 'Party with Lucy in the sky with diamonds']}
FP feedback reasoning start
{
"reason": "The model is overfitting on the word 'drug' and classifying any text that contains it as harmful, regardless of context. In the provided misclassified sample, 'drug' is mentioned in a medical context, which is safe.",
"generated_data": [
"My doctor prescribed me a drug",
"I need to pick up my prescription drug from the pharmacy",
"The drug prescribed by my doctor has helped with my symptoms",
"My pharmacist gave me advice on how to take my new drug",
"The drug my doctor recommended

In [7]:
# step 2: Edit the description according to the reasoning if necessary
new_description = client.edit_description(guideline, feedback_reasoning_result)
guideline_edited = guideline.copy()
guideline_edited["harmful_description"] = new_description["harmful_description"]
new_description


FN description edited: The current harmful description only identifies explicit references to drugs and doesn't account for indirect references or slang. Expanding the description to include these aspects will improve the model's ability to correctly classify these types of texts. The safe description is also updated to specify that medical or legal references to drugs are not considered harmful.
FP description edited: The original harmful description was too broad and included any mention of the word 'drug', regardless of context. The safe description has been expanded to specifically include cases where 'drug' is used in a medical, pharmaceutical, or legal context, which were previously misclassified as harmful.


{'edited': True,
 'harmful_description': "Content containing explicit mentions or indirect references, including common slang terms, related to drugs. This includes phrases where words such as 'drug' are used to refer to illegal substances.",
 'safe_description': "Content that does not contain any explicit mentions or indirect references to illegal drugs. This includes cases where words such as 'drug' are used in a medical, pharmaceutical, or legal context."}

In [8]:
# step 3: Create more related samples for training
augmented_results = {"FP": [], "FN": []}

if feedback_reasoning_result["FN"] != None:
    for i in feedback_reasoning_result["FN"]["generated_data"]:
        augmented_result = client.augmentation_fn(guideline_edited, [i])
        augmented_results["FN"]+=augmented_result
if feedback_reasoning_result["FP"] != None:
    for i in feedback_reasoning_result["FP"]["generated_data"]:
        augmented_result = client.augmentation_fp(guideline_edited, [i])
        augmented_results["FP"]+=augmented_result

In [9]:
# step 4: Use the edited description and augmented data to retrain the model
print(f"Whether the decritpion is edited and the edited result:\n {new_description['edited']}")
print(f"The content below should be added as the harmful samples: \n {augmented_results['FN']}")
print(f"The content below should be added as the normal samples: \n {augmented_results['FP']}")

Whether the decritpion is edited and the edited result:
 True
The content below should be added as the harmful samples: 
 ["The party this weekend is going to be epic, we're bringing the snow.", "I've got the best candy if you're looking for a good time.", "If you're feeling down, I know a grass that will lift your spirits.", 'The white lady is going to make an appearance at the gathering.', "Let's catch some clouds and fly together.", "I've got some magic beans that can take us to cloud nine.", 'Fancy a trip to the moon? I have the rocket fuel.', 'Ever dreamt of flying? I can give you wings with my special candy.', 'The pixie dust I have can make you dance with the stars.', "Let's ride the rainbow with my secret potion.", "Let's meet up and pop some pills tonight.", 'The white powder takes me to nirvana.', "I've got some good stuff, the kind that makes you fly.", 'Time to smoke some grass and drift away.', "Got them magic mushrooms? Let's get trippy.", "Let's do a snowfall party tonig